In [4]:
import datasets
import pandas as pd
from pathlib import Path
from PIL import Image
from transformers import AutoTokenizer
from transformers import Blip2Processor, Blip2VisionModel, Blip2QFormerModel, Blip2QFormerConfig, Blip2ForConditionalGeneration
from transformers import AutoProcessor, Blip2ForConditionalGeneration

import torch

In [5]:
# !pip install peft
# !pip install bitsandbytes

In [7]:
!pip install bitsandbytes

In [8]:
import os
import bitsandbytes as bnb


In [9]:
from peft import LoraConfig, get_peft_model,LoftQConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,)

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", #TODO put back in "load_in_8bit" for model
    device_map={"": 0},
    trust_remote_code=True,
    quantization_config=bnb_config,
)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
from peft import LoraConfig, get_peft_model, LoftQConfig

# loftQConfig = LoftQConfig(loftq_bits=4)
# Let's define the LoraConfig
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 5,242,880 || all params: 3,749,922,816 || trainable%: 0.13981301102065136


In [60]:
images_path = list(Path("/root/Datasets/Images").glob("*.jpg"))
labels = pd.read_csv("/root/Datasets/preprocess.csv")
images = [Image.open(str(images_path[0].parent / path)) for path in labels['image']]

dataset = datasets.Dataset.from_dict({"image": images, "text": labels['caption']})
dataset = dataset.train_test_split(test_size=0.1, seed=42)

[PosixPath('/content/train2017/000000321333.jpg'), PosixPath('/content/train2017/000000335328.jpg'), PosixPath('/content/train2017/000000320425.jpg'), PosixPath('/content/train2017/000000132116.jpg'), PosixPath('/content/train2017/000000119038.jpg'), PosixPath('/content/train2017/000000041872.jpg'), PosixPath('/content/train2017/000000513688.jpg'), PosixPath('/content/train2017/000000228144.jpg'), PosixPath('/content/train2017/000000346905.jpg'), PosixPath('/content/train2017/000000365745.jpg'), PosixPath('/content/train2017/000000481404.jpg'), PosixPath('/content/train2017/000000117492.jpg'), PosixPath('/content/train2017/000000158548.jpg'), PosixPath('/content/train2017/000000114871.jpg'), PosixPath('/content/train2017/000000577584.jpg'), PosixPath('/content/train2017/000000314294.jpg'), PosixPath('/content/train2017/000000231527.jpg'), PosixPath('/content/train2017/000000415238.jpg'), PosixPath('/content/train2017/000000081738.jpg'), PosixPath('/content/train2017/000000463918.jpg'),

FileNotFoundError: [Errno 2] No such file or directory: 'content/train2017/000000373716.jpg'

In [46]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = item["text"]
        return encoding

def collate_fn(batch):
    # pad the input_ids and attention_mask
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

In [ ]:
train_dataset = ImageCaptioningDataset(dataset['train'], processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)

test_dataset = ImageCaptioningDataset(dataset['test'], processor)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)

In [ ]:
import torch
from tqdm import tqdm
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()

for epoch in range(10):
    training_loss = 0
    print("Epoch:", epoch)
    for idx, batch in tqdm(enumerate(train_dataloader)):
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device)
        outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        labels=input_ids)

        loss = outputs.loss
        training_loss += loss.item()
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
    print(training_loss / len(train_dataloader))

In [ ]:
im = dataset['test'].select([290])['image']

In [ ]:
pixel_values = processor(im, return_tensors="pt").to(device).pixel_values
model.eval()

outputs = model.generate(
    pixel_values=pixel_values,
    max_length=32,
    num_beams=4,
    no_repeat_ngram_size=3,
    
    )

In [ ]:
generated_caption = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print(generated_caption)